# IMPORTS

In [ ]:
import numpy, scipy, pandas as pd
from scipy.io.arff import loadarff
import matplotlib.pyplot as plt
import numpy as np

# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


from IPython.display import clear_output


# LOADING DATA

In [ ]:
motor_train_f = "DataSets/MotorImagery/MotorImagery_TRAIN.arff"
motor_test_f = "DataSets/MotorImagery/MotorImagery_TEST.arff"

In [ ]:
# IT TAKES AROUND 1-2 MINUTES TO LOAD THE DATASET

motor_train, motor_train_meta = loadarff(motor_train_f)
df = pd.DataFrame(motor_train)

dataa = df.relationalAtt
tagg = df.motion

tagg = tagg.replace(b'finger', 1)
tagg = tagg.replace(b'tongue', 0)

In [ ]:
moto_test, moto_test_meta = loadarff(motor_test_f)
df_test = pd.DataFrame(moto_test)

dataa_test = df_test.relationalAtt
tagg_test = df_test.motion

tagg_test = tagg_test.replace(b'finger', 1)
tagg_test = tagg_test.replace(b'tongue', 0)

In [ ]:
x_train = dataa
y_train = tagg

x_test = dataa_test
y_test = tagg_test

# PRE PROCESSING

In [ ]:
mega_dataa = []

In [ ]:
# print(len(mega_dataa))
def make_obs_onechannel_time(dataa, n, tot_obss): # SHOULD RETURN 278 X 3000 MATRIX
    _mega_dataa = []
    for i in range(tot_obss):

        obs = []
        for ch in range(1):
            ch = n
            chh = []
            for j in range(3000):
                chh.append(dataa[i][f'att{j}'][ch])
                
            obs = chh
        _mega_dataa.append(obs)

    return _mega_dataa 

In [ ]:
def flat(lists):
    return [item for sublist in lists for item in sublist]

In [ ]:
def squeeze(obs_time_matrix): # SHOULD RETURN 1 X 278 MATRIX
    # obs_time_matrix is 278 x 200 matrix 
    squeezed = []
    for obs in obs_time_matrix:
        squeezed.append(np.average(np.array(obs))) # AVERAGING THE TIME
    return np.array(squeezed) # WILL RETURN 1 X 278 MATRIX


In [ ]:
dataa.shape, dataa[0].shape, dataa[0][0].shape, dataa_test.shape, dataa_test[0].shape, dataa_test[0][0].shape

In [ ]:
# THIS CELL MAY TAKE 7-8 MINS OR MORE

mega_dataa_ch_wis = []
mega_dataa_ch_wis_test = []
# ica = FastICA(n_components=200)
pca = PCA(n_components=50)

for i in range(64): # FOR EACH CHANNEL WE ARE FINDING 50 TIME COMPONENTs
    clear_output(wait=True)
    print(f"reading {i}")

    # CONVERTING 278 X 3000 -> 278 X 50 MATRIX
    X_pca = pca.fit_transform(make_obs_onechannel_time(dataa, i, 278)) # CHOOSE 50 BEST COLUMN, SO MATRIX = 278 X 50

    X_pca_test = pca.transform(make_obs_onechannel_time(dataa_test, i, 100)) # CHOOSE 10 BEST COLUMN, SO MATRIX = 278 X 10

    mega_dataa_ch_wis.append(X_pca)
    mega_dataa_ch_wis_test.append(X_pca_test)

print(len(mega_dataa_ch_wis), len(mega_dataa_ch_wis[0])) # SHOULD BE 64 X 278

In [ ]:
print(len(mega_dataa_ch_wis), len(mega_dataa_ch_wis[0]), len(mega_dataa_ch_wis[0][0]))

In [ ]:
mn = []
for i in range(278): # LENGTH OF TRAIN
    mn_l = []
    for j in range(64): # LENGTH O CHANNELS
        for k in range(50): # SIZE USED IN PCA
            mn_l.append(mega_dataa_ch_wis[j][i][k])
    mn.append(mn_l)
print(len(mn), len(mn[0]))

In [ ]:
mn_test = []
for i in range(100): # LENGTH OF TEST
    mn_l = []
    for j in range(64): # LENGTH OF CHANNELS
        for k in range(50): # SIZE USED IN PCA
            mn_l.append(mega_dataa_ch_wis_test[j][i][k])
    mn_test.append(mn_l)
print(len(mn_test), len(mn_test[0]))

In [ ]:
x_train = mn
x_test = mn_test


# CHECKING CLASSIFIERS

### APPLYING RANDOM FOREST CLASSIFIER

In [ ]:
pca = PCA(n_components=20)

X_train_pca = pca.fit_transform(mn) # Train a random forest classifier on the transformed data

rfc = RandomForestClassifier(max_depth=10, n_estimators=200)


rfc.fit(mn, y_train) # Transform the test data using the fitted PCA object

X_test_pca = pca.transform(mn_test) # Make predictions on the test set

y_pred = rfc.predict(mn_test) # Compute the accuracy of the model


rfc_accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {rfc_accuracy*100:0.3f}')

# MAX ACCURACY OBSERVED WAS 65, AVERAGE WAS AROUND 58-61

### APPLYING ADA BOOST CLASSIFIER

In [ ]:
ada_c = AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=0)
ada_model = ada_c.fit(mn, y_train)
y_pred = ada_model.predict(mn_test)

ada_accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {ada_accuracy*100}')


### APPLYING SVC

In [ ]:
svc_model = SVC(C=.1, kernel='poly', gamma=1)
svc_model.fit(mn, y_train)
 
prediction = svc_model.predict(mn_test)

svc_score_ = svc_model.score(mn_test, y_test)
# print(svc_model.score(x_train, y_train))
print(f"Accuracy : {svc_score_*100}")

### APPLYING ADA-BOOST WITH SVC AS BASE ESTIMATOR

In [ ]:
# base_estimator = MLPClassifier(hidden_layer_sizes=(5,), max_iter=1000, random_state=0)
base_estimator = SVC(probability=True, kernel='poly')
ada_c = AdaBoostClassifier(estimator=base_estimator, n_estimators=100, random_state=3, learning_rate=0.5)

# clf = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100, random_state=0)

# clf.fit(mn, y_train)

# print(clf.predict(mn_test))

ada_model = ada_c.fit(mn, y_train)
y_pred = ada_model.predict(mn_test)

ada2_accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {ada2_accuracy*100}')

In [ ]:
scores = [rfc_accuracy, ada_accuracy,svc_score_, ada2_accuracy ]
strr = "rfc_accuracy, ada_accuracy,svc_score_, ada2_accuracy "
x_ax = [i for i in strr.split(",")]
x = [i for i in range(len(x_ax))]

scores = np.array(scores)*100
plt.figure(figsize=(10, 10))
# plt.ylim([-1, 100])
plt.xticks(x, x_ax)
plt.plot(x, scores)
plt.scatter(x, scores)
for i, txt in enumerate(x_ax):
    plt.annotate(txt, (x[i]-0.2 ,scores[i]+0.1))
plt.grid()